In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Apathy

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Apathy
data = pd.read_csv('Apathy_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Apathy


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,MDS_Apathy_Category,cg19958696,cg05962325,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.010314,0.057597,...,0.751058,0.901978,0.580283,0.548182,0.725312,0.092118,0.723395,0.129795,0.017529,0.026699


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','MDS_Apathy_Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg19958696  cg05962325  cg15093662  cg04873221  cg05903298  cg19020436  \
0    0.010314    0.057597    0.020549    0.015659    0.011741    0.014318   
1    0.014601    0.068728    0.019329    0.021747    0.012923    0.017399   

   cg14994056  cg05973813  cg18337422  cg11738745  ...  cg19795267  \
0    0.035177    0.014771    0.013507    0.010626  ...    0.751058   
1    0.033939    0.020282    0.017160    0.012127  ...    0.630319   

   cg16922840  cg19628277  cg26480988  cg01673347  cg12994818  cg09008531  \
0    0.901978    0.580283    0.548182    0.725312    0.092118    0.723395   
1    0.851560    0.521620    0.495925    0.830013    0.076462    0.728590   

   cg16463148  cg04021430  cg04863888  
0    0.129795    0.017529    0.026699  
1    0.246918    0.039718    0.024071  

[2 rows x 3564 columns]


In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6413043478260869

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	3564


BorutaPy(estimator=RandomForestClassifier(n_estimators=844,
                                          random_state=RandomState(MT19937) at 0x1498AE2BC40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1498AE2BC40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
1030,cg01081395,2
1996,cg10013170,3
811,cg04856117,3


In [10]:
Top50.to_csv ('Apathy_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Apathy_Boruta_All.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Apathy_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11932\3868420342.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg14824670,cg18299793,cg08556479,cg10164191,cg05064739,cg03476938,cg26840770,cg12049462,cg23804054,cg01589384,...,cg07897734,cg11875268,cg23203206,cg06632027,cg02553661,cg15567507,cg19458274,cg25324991,cg19628277,APPRDX
0,0.312659,0.923536,0.864220,0.511121,0.88989,0.611944,0.109552,0.208430,0.290418,0.038556,...,0.890719,0.953648,0.621678,0.043348,0.947492,0.906558,0.018417,0.693119,0.580283,2
1,0.312060,0.885384,0.866326,0.604161,0.85152,0.549004,0.058578,0.131758,0.215196,0.042105,...,0.842938,0.956079,0.850858,0.029793,0.929744,0.824857,0.018518,0.599772,0.521620,1


# Gastro

In [12]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro
data = pd.read_csv('Gastro_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg12522722,cg11255513,cg09615521,cg12658517,cg02150941,cg22992837,cg02699881,cg05225390,cg01481208,cg08634588
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.89241,0.015756,...,0.909955,0.043985,0.054816,0.904326,0.135138,0.742557,0.029678,0.079017,0.894593,0.902949


In [13]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','SCOPA_Gastro_Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg03441945  cg21094410  cg22926093  cg25025609  cg16704802  cg06228737  \
0    0.892410    0.015756    0.011101    0.018491    0.020476    0.962482   
1    0.908813    0.011350    0.012197    0.022372    0.018128    0.945195   

   cg00220102  cg19016494  cg12247200  cg24658413  ...  cg12522722  \
0    0.677222    0.029560    0.929821    0.899519  ...    0.909955   
1    0.730494    0.017911    0.921435    0.815257  ...    0.900398   

   cg11255513  cg09615521  cg12658517  cg02150941  cg22992837  cg02699881  \
0    0.043985    0.054816    0.904326    0.135138    0.742557    0.029678   
1    0.043525    0.043829    0.899376    0.078902    0.754703    0.030224   

   cg05225390  cg01481208  cg08634588  
0    0.079017    0.894593    0.902949  
1    0.075161    0.909315    0.903716  

[2 rows x 34004 columns]


In [14]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [15]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6413043478260869

In [16]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100


BorutaPy finished running.

Iteration: 	30 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	34003


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1498B13F640),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1498B13F640, verbose=1)

In [17]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [18]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [19]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
32317,cg16199144,1
20524,cg21921473,2
788,cg25774148,2


In [20]:
Top50.to_csv ('Gastro_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Gastro_Boruta_All.csv', index = False)

In [21]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Gastro_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11932\3247414631.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg12452675,cg25774148,cg04926619,cg02744123,cg11351331,cg07250622,cg21043558,cg11697588,cg12510028,cg14841337,...,cg22152677,cg06734098,cg12220605,cg00875272,cg13468400,cg08096908,cg17223615,cg16199144,cg19371224,APPRDX
0,0.047168,0.940005,0.627342,0.014085,0.895471,0.058003,0.638077,0.051262,0.037114,0.027298,...,0.932195,0.340984,0.117280,0.032753,0.895386,0.937764,0.822963,0.033485,0.895183,2
1,0.040326,0.915349,0.632705,0.013944,0.898821,0.032852,0.667449,0.076483,0.029810,0.021279,...,0.927876,0.050031,0.082447,0.026473,0.812987,0.905542,0.814484,0.027345,0.751572,1


# Letter Number

In [22]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LetterNumber
data = pd.read_csv('LetterNumber_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LetterNumber


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,LetterNumber_Category,cg20815302,cg25795414,...,cg27475074,cg02546039,cg25870071,cg26765773,cg22019496,cg02429154,cg05411695,cg13202877,cg17697230,cg05687264
0,200973410159_R03C01,3000,2,2,0,69.1,4,1,0.91579,0.899909,...,0.028559,0.920549,0.469715,0.112805,0.892764,0.925107,0.070574,0.165807,0.918808,0.928897


In [23]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','LetterNumber_Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg20815302  cg25795414  cg00959375  cg07781847  cg05138383  cg27394794  \
0    0.915790    0.899909    0.804116    0.796131    0.740029    0.838002   
1    0.907957    0.868910    0.752844    0.827413    0.771393    0.766283   

   cg12443990  cg19788329  cg22178513  cg04167299  ...  cg27475074  \
0    0.822036    0.813254    0.745601    0.886094  ...    0.028559   
1    0.791932    0.785104    0.723989    0.861925  ...    0.049168   

   cg02546039  cg25870071  cg26765773  cg22019496  cg02429154  cg05411695  \
0    0.920549    0.469715    0.112805    0.892764    0.925107    0.070574   
1    0.906285    0.458602    0.068597    0.859078    0.907987    0.036712   

   cg13202877  cg17697230  cg05687264  
0    0.165807    0.918808    0.928897  
1    0.177144    0.927030    0.911733  

[2 rows x 73185 columns]


In [24]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [25]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6630434782608695

In [26]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	73185


BorutaPy(estimator=RandomForestClassifier(n_estimators=3825,
                                          random_state=RandomState(MT19937) at 0x1498B13F940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1498B13F940, verbose=1)

In [27]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [28]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [29]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
27627,cg23937960,2
5711,cg24467842,2
42306,cg06709766,3


In [30]:
Top50.to_csv ('LetterNumber_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('LetterNumber_Boruta_All.csv', index = False)

In [31]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('LetterNumber_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11932\226587430.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg02471954,cg26657920,cg00260798,cg04009197,cg03245734,cg03424478,cg24467842,cg21747188,cg13312668,cg10619653,...,cg27542525,cg10322510,cg18967713,cg11802732,cg12220605,cg12015492,cg00811166,cg12046288,cg19371224,APPRDX
0,0.928420,0.876827,0.924389,0.898288,0.042934,0.028517,0.830469,0.025714,0.902683,0.935868,...,0.329920,0.967393,0.015011,0.039124,0.117280,0.033413,0.887979,0.017441,0.895183,2
1,0.853669,0.836352,0.885679,0.850314,0.047562,0.030953,0.874708,0.064156,0.858468,0.906738,...,0.347469,0.967266,0.019488,0.038867,0.082447,0.039214,0.840791,0.019627,0.751572,1


# Lexical Fluency

In [32]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LexicalFluency
data = pd.read_csv('LexicalFluency_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LexicalFluency


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Lexical_Fluency_Category,cg02315971,cg24133836,...,cg12092065,cg10388349,cg11918957,cg26583598,cg11023075,cg02097667,cg21616723,cg27335855,cg18259253,cg15404040
0,200973410159_R03C01,3000,2,2,0,69.1,4,1,0.030109,0.016106,...,0.953091,0.392056,0.913402,0.913637,0.897198,0.871237,0.913624,0.65186,0.852542,0.909412


In [33]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','Lexical_Fluency_Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg02315971  cg24133836  cg03286969  cg11738782  cg25741731  cg24629744  \
0    0.030109    0.016106    0.019371    0.884151    0.026326    0.868373   
1    0.028697    0.017000    0.018631    0.871526    0.026148    0.866396   

   cg23836745  cg11685489  cg06081147  cg18057762  ...  cg12092065  \
0    0.905809    0.825562    0.691096    0.045994  ...    0.953091   
1    0.922285    0.881489    0.647436    0.035721  ...    0.924513   

   cg10388349  cg11918957  cg26583598  cg11023075  cg02097667  cg21616723  \
0    0.392056    0.913402    0.913637    0.897198    0.871237    0.913624   
1    0.370289    0.902610    0.914253    0.881206    0.834363    0.912120   

   cg27335855  cg18259253  cg15404040  
0     0.65186    0.852542    0.909412  
1     0.69253    0.839574    0.867794  

[2 rows x 85915 columns]


In [34]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [35]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6808510638297872

In [36]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100


BorutaPy finished running.

Iteration: 	33 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	85914


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1498B13F840),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1498B13F840, verbose=1)

In [37]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [38]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [39]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
38024,cg21507882,1
77927,cg09304381,2
45363,cg19984257,3


In [40]:
Top50.to_csv ('LexicalFluency_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('LexicalFluency_Boruta_All.csv', index = False)

In [41]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('LexicalFluency_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11932\1700292862.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg26354221,cg14296039,cg10475683,cg23254463,cg06273051,cg18823302,cg15355235,cg07027506,cg02479278,cg25816115,...,cg15593930,cg13359134,cg07667731,cg09304381,cg23169811,cg26869617,cg21585627,cg02618319,cg09670616,APPRDX
0,0.918085,0.873012,0.062533,0.919921,0.543591,0.944782,0.633212,0.645067,0.677445,0.875295,...,0.019946,0.016097,0.016602,0.981877,0.933661,0.923248,0.042639,0.981089,0.017773,2
1,0.924711,0.835527,0.044298,0.921331,0.492912,0.890833,0.027949,0.578137,0.641669,0.815073,...,0.027932,0.014686,0.013778,0.960567,0.931264,0.906871,0.057217,0.977055,0.031018,1


# Trail Making B

In [42]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB
data = pd.read_csv('TrailMakeB_with_MethylomeDetails.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Trail_Making_B_Category,cg02315971,cg24133836,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.030109,0.016106,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.04635,0.904242,0.021033,0.813738


In [43]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','Trail_Making_B_Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg02315971  cg24133836  cg03286969  cg11738782  cg25741731  cg24629744  \
0    0.030109    0.016106    0.019371    0.884151    0.026326    0.868373   
1    0.028697    0.017000    0.018631    0.871526    0.026148    0.866396   

   cg23836745  cg11685489  cg06081147  cg18057762  ...  cg00956423  \
0    0.905809    0.825562    0.691096    0.045994  ...    0.148515   
1    0.922285    0.881489    0.647436    0.035721  ...    0.109870   

   cg14462343  cg07481734  cg22394535  cg13044444  cg23223756  cg16870358  \
0    0.036822    0.053052    0.044051    0.040994    0.047501    0.046350   
1    0.056582    0.064137    0.053802    0.083556    0.042638    0.041733   

   cg04754776  cg00154746  cg06155771  
0    0.904242    0.021033    0.813738  
1    0.896521    0.033285    0.753096  

[2 rows x 61918 columns]


In [44]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [45]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.5909090909090909

In [46]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100


BorutaPy finished running.

Iteration: 	30 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	61917


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1498B13F540),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1498B13F540, verbose=1)

In [47]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [48]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [49]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
19521,cg10729106,1
54689,cg19810426,2
17648,cg13167828,3


In [50]:
Top50.to_csv ('TrailMakeB_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('TrailMakeB_Boruta_All.csv', index = False)

In [51]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('TrailMakeB_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11932\958766410.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg18439474,cg26354221,cg01746044,cg17898737,cg12153529,cg02109003,cg13122031,cg20721322,cg11492856,ch.22.138449R,...,cg15469205,cg09495953,cg16620860,cg19810426,cg10425946,cg12417702,cg27497687,cg16356224,cg25911279,APPRDX
0,0.790590,0.918085,0.021957,0.923107,0.026793,0.717723,0.910175,0.255404,0.921944,0.070647,...,0.656929,0.812500,0.624056,0.337330,0.017390,0.069469,0.037778,0.148380,0.035425,2
1,0.818168,0.924711,0.051224,0.916143,0.032767,0.639496,0.813401,0.208455,0.911869,0.076957,...,0.865868,0.907085,0.582799,0.293279,0.051054,0.030323,0.033875,0.179434,0.031463,1


#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
